In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tensorflow-addons
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

No GPU was detected. LSTMs and CNNs can be very slow without a GPU.


## Char-RNN (stateless RNN)

Guessing the next character at a time.

### Splitting a seq into batches of shuffled windows

Don't know what this is for yet.

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

n_steps=5
dataset = tf.data.Dataset.from_tensor_slices(tf.range(15))
dataset = dataset.window(n_steps, shift=2, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(n_steps))
dataset = dataset.shuffle(10).map(lambda window: (window[:-1], window[1:]))
dataset = dataset.batch(3).prefetch(1)
for index, (X_batch, Y_batch) in enumerate(dataset):
    print("_" * 20, "Batch", index, "\nX_batch")
    print(X_batch.numpy())
    print("=" * 5, "\nY_batch")
    print(Y_batch.numpy())

____________________ Batch 0 
X_batch
[[6 7 8 9]
 [2 3 4 5]
 [4 5 6 7]]
===== 
Y_batch
[[ 7  8  9 10]
 [ 3  4  5  6]
 [ 5  6  7  8]]
____________________ Batch 1 
X_batch
[[ 0  1  2  3]
 [ 8  9 10 11]
 [10 11 12 13]]
===== 
Y_batch
[[ 1  2  3  4]
 [ 9 10 11 12]
 [11 12 13 14]]


### Loading the data and prepping the Dataset

In [3]:
shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

1122304/1115394 [==============================] - 0s 0us/step


In [4]:
print (shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [5]:
"".join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

Fitting a tokenizer to the text, this is to encode the characters to integers.

In [6]:
#char_level = True b/c default is word level tokenizing
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [7]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [8]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [9]:
max_id = len(tokenizer.word_index) # num of distinct characters
dataset_size = tokenizer.document_count # total num of characters

In [10]:
#subtracting 1 so that Id's start at 0...
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

### Split a Sequential Dataset

To avoid any overlap, want a gap between the sets. The text talks about the difficulty of splitting a time series. There is an assumption that is made when splitting across time, that the patterns in the past will hold inot the future, which works in some places but not others. When we can make the assumption, the time series is called *stationary*. When its not, it looks like things stock market data which is quite volatile and patterns are taken advanatge of right when their found. 

To make sure that the time series is stationary, you can plot the models errors on a the validation set across time: if it performs better at the start of the validation set than the end, then it might not be stationary and should train the model on a shorter time span.

We take 90% of the data to train on below, and create a Dataset that will return each character 1by1 from the set:

In [15]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

### Chopping Sequential Dataset into Multiple Windows

There are over 1 million characters in the seq now and cant just throw it into an RNN as a one long seq. Using the dataset's `window()` method, will convert seq into smaller windows of text. The instances will be short, and the RNN will unroll over the length of these substrings, whhich is called *truncated backpropagation*.

In [16]:
n_steps = 100
window_length = n_steps + 1 #target = input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

`window()` will create non-overlapping windows by default, so `shift=1` will make it so that the first batch is 0 to 100, the second 1 to 101, and so on. To make sure that there are exactly 101 characters per batch, `drop_remainder=True` so that the last 100 windows dont have 100, 99,98...2,1 characters in them. 

`window()` method creates a dataset that contains windows, each of which is also a dataset, so its nested datasets. Useful when trying to call the datasets methods (like shuffle or batch), but can't use it for training since model expects tensors as inputs. `flat_map()` will convert the nested dataset into one flat one.

Throwing a function into `flat_map()` can transform it before flattening, below we are making batches size (n_steps + 1) in the flatten dataset.

In [17]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [18]:
np.random.seed(42)
tf.random.set_seed(42)

In [19]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:,:-1], windows[:,1:]))

In [20]:
dataset = dataset.map(lambda 
                      X_batch,
                      Y_batch: (tf.one_hot(X_batch, depth=max_id), 
                                Y_batch))


In [21]:
dataset = dataset.prefetch(1)

In [24]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


### Creating and Training the Model

In [25]:
# this is slow as hell, two hours per epoch
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None,max_id],
                    dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, dropout=0.2,
                    recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, steps_per_epoch = train_size // batch_size,
                   epochs=10)

Epoch 1/10
   62/31370 [..............................] - ETA: 2:40:01 - loss: 3.1292

KeyboardInterrupt: 

### Generating Text

If the model is trained...

In [26]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [28]:
#X_new = preprocess(["How are yo"])
#Y_pred = model.predict_classes(X_new)
#tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

#outputs 'u'

Now we want to generate text, dont want to find the last character and then refeed that to find the next and so on, b/c it tends to repeat itseelf. 

So we pick the next character randomly, which will diversify the text. The `categorical()` function samples random class indices, given the class log probabilites (logits). Can also set the temperature, where closer to 0 gives high prob characters, higher temperatures will give each character an equal prob.

In [29]:
tf.random.set_seed(42)

tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], 
                      num_samples=40).numpy()

array([[0, 1, 0, 2, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 2, 1, 0, 2, 1,
        0, 1, 2, 1, 1, 1, 2, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 2]])

In [30]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [31]:
tf.random.set_seed(42)

#next_char("How are yo", temperature=1)

#outputs 'u'

In [32]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [34]:
tf.random.set_seed(42)

# print(complete_text("t", temperature=0.2))

'''the belly the charges of the other words
and belly '''

# print(complete_text("t", temperature = 1))

'''thing! they know't.

biondello:
for you are the own'''

# print(complete_text("t", temperature=2))

'''th no cyty
use ffor was firive this toighingaber; b'''

'th no cyty\nuse ffor was firive this toighingaber; b'

## Stateful RNNs

Stateless RNN's at each training iteration start with a hidden state full of zeros, updates them at each time step, then throws them out at the last time step. A stateful RNN preserves the hidden states across batches.